# Darcy equation: exercise 4

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k=I$ the matrix permeability and $f$ a scalar source term, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = 0\\
\nabla \cdot {q} = f
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ \nu \cdot q = 0 \text{ on } \partial \Omega$$
Where $f = \pm 1$ represents two wells, one with $+1$ and one with $-1$, set on the two corners of the domain.
The problem clearly does not have a unique solution and we need to design a strategy to make it solvable.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, since we will use a Raviart-Thomas approximation for ${q}$ we are restricted to simplices. In this example we consider a 2-dimensional grid.

In [ ]:
mesh_size = 0.05
# creation of the grid
sd = pg.unit_grid(2, mesh_size, as_mdg=False)
# compute the geometrical properties of the grid
sd.compute_geometry()

# pp.plot_grid(sd, info="c", alpha=0, fig_size=(20, 20))

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [18]:
key = "flow"

# the cells of the two wells, specific for the current grid
well_in = 878
well_out = 865

# declare the discretization objects, useful to setup the data
rt0 = pg.RT0(key)
p0 = pg.PwConstants(key)

# set up the data for the flow problem
data = {}

# set the permeability
perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
parameters = {
    "second_order_tensor": perm,
}
pp.initialize_data(sd, data, key, parameters)

# set up the wells
scalar_source = np.zeros(sd.num_cells)
scalar_source[well_in] = 1
scalar_source[well_out] = -1

# set for all the boundary essential condition
ess_q_dofs = np.zeros(rt0.ndof(sd), dtype=bool)
ess_p_dofs = np.zeros(p0.ndof(sd), dtype=bool)
bc_ess = np.hstack((ess_q_dofs, ess_p_dofs))

Once the data are assigned to the grid, we construct the matrices. In particular, the linear system associated with the equation is given as
$$
\left(
\begin{array}{cc} 
M & -B^\top\\
B & 0
\end{array}
\right)
\left(
\begin{array}{c} 
q\\ 
p
\end{array}
\right)
=\left(
\begin{array}{c} 
0\\ 
f
\end{array}
\right)
$$<br>
where $p_{\partial}$ is the vector associated to the pressure boundary contions. To construct the saddle-point problem, we rely on the `scipy.sparse` function `bmat`. Once the matrix is created, we also construct the right-hand side containing the boundary conditions.

In [19]:
# construct the local matrices
mass_rt0 = rt0.assemble_mass_matrix(sd, data)
mass_p0 = p0.assemble_mass_matrix(sd, data)
div = mass_p0 @ rt0.assemble_diff_matrix(sd)

# assemble the saddle point problem
spp = sps.block_array(
    [
        [mass_rt0, -div.T],
        [div, None],
    ],
    format="csc",
)

# get the degrees of freedom for each variable
dof_p, dof_q = div.shape

However, the matrix `spp` is singular but we can impose that the pressure has zero average
$$
    \int_\Omega p = 0 \quad \Rightarrow \quad \sum_i p_i = 0
$$
since our pressure degrees of freedom already includes the measure of the cells. We can use a Lagrange multiplier to impose this constraint, we add a new line to the system and its corresponding (anti)transpose.

In [20]:
# construct the contraint
cons = np.ones((1, dof_q + dof_p))
cons[0, dof_q:] = np.ones(dof_p)

# add the constraint
spp = sps.block_array(
    [
        [spp, -cons.T],
        [cons, None],
    ],
    format="csc",
)

# assemble the right-hand side
rhs = np.zeros(dof_q + dof_p + 1)
rhs[dof_q:-1] += scalar_source

We need to solve the linear system and extract the two solutions $q$ and $p$, by remembering to discard the last row used for the Lagrange multiplier.

In [21]:
# fix the bc
bc = np.zeros_like(rhs, dtype=bool)
bc[:-1] = bc_ess

# solve the problem
ls = pg.LinearSystem(spp, rhs)
ls.flag_ess_bc(bc, np.zeros_like(rhs))
x = ls.solve()[:-1]

# extract the variables
q = x[:dof_q]
p = x[-dof_p:]

Since the computed $q$ is one value per facet of the grid, for visualization purposes we project the flux in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [22]:
# post process variables
proj_q = rt0.eval_at_cell_centers(sd)
cell_q = (proj_q @ q).reshape((3, -1))
cell_p = p0.eval_at_cell_centers(sd) @ p

save = pp.Exporter(sd, "sol", folder_name="ex4")
save.write_vtu([("cell_p", cell_p), ("cell_q", cell_q)])